In [1]:
import pandas as pd

In [2]:
file = 'BB_20190227_RE_data_chronostrat_combine_test.xlsx'
data = pd.ExcelFile(file)

In [3]:
sheets = data.sheet_names
sheets

['Chronostrat', 'RE data']

In [4]:
chrono = data.parse(sheets[0])
redata = data.parse(sheets[1])

In [5]:
#Show missing entries
def num_missing(x):
  return sum(x.isnull())

In [6]:
#Applying per column:
print ("Missing values per column:")
print (chrono.apply(num_missing, axis=0)) #axis=0 column

#Applying per row:
print ("\nMissing values per row:")
print (chrono.apply(num_missing, axis=1).head()) #axis=1 row

#Applying per column:
print ("Missing values per column:")
print (redata.apply(num_missing, axis=0)) #axis=0 column

#Applying per row:
print ("\nMissing values per row:")
print (redata.apply(num_missing, axis=1).head()) #axis=1 row

Missing values per column:
Well             0
Period           0
Top_depth_m      0
Base_depth_m     0
Thickness_m      0
Top_age_Ma       0
Bottom_age_Ma    0
dtype: int64

Missing values per row:
0    0
1    0
2    0
3    0
4    0
dtype: int64
Missing values per column:
Well                    0
Depth m RKB             0
Formation (NPD)         0
Type of measurement     0
samplith                0
tmax                    7
s1                      0
s2                      0
s3                     31
toc                     0
HI                      0
pi                      0
company_name            0
dtype: int64

Missing values per row:
0    0
1    0
2    0
3    0
4    0
dtype: int64


In [7]:
chrono.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 7 columns):
Well             105 non-null object
Period           105 non-null object
Top_depth_m      105 non-null int64
Base_depth_m     105 non-null int64
Thickness_m      105 non-null int64
Top_age_Ma       105 non-null float64
Bottom_age_Ma    105 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 5.8+ KB


In [8]:
merge = pd.merge(chrono,redata)

In [9]:
len(merge)

10821

In [10]:
print(len(redata[redata['Well']=='6406/2-1']))
print(len(redata[redata['Well']=='6406/2-2']))
print(len(redata[redata['Well']=='6406/3-1']))

137
83
83


In [11]:
print(len(merge[merge['Well']=='6406/2-1']))
print(len(merge[merge['Well']=='6406/2-2']))
print(len(merge[merge['Well']=='6406/3-1']))

5343
2905
2573


In [12]:
print(len(chrono[ chrono['Well']=='6406/2-1']))
print(len(chrono[ chrono['Well']=='6406/2-2']))
print(len(chrono[ chrono['Well']=='6406/3-1']))

39
35
31


In [13]:
merge.head()

,Well,Period,Top_depth_m,Base_depth_m,Thickness_m,Top_age_Ma,Bottom_age_Ma,Depth m RKB,Formation (NPD),Type of measurement,samplith,tmax,s1,s2,s3,toc,HI,pi,company_name
0,6406/2-1,QUATERNARY,302,1240,938,0.0,65.0,2510.000,SPRINGAR FM,DC,SH/CLYST,356.0,2.20,4.61,2.14,1.15,401,0.32,GEOLABNOR
1,6406/2-1,QUATERNARY,302,1240,938,0.0,65.0,2520.000,SPRINGAR FM,DC,SH/CLYST,362.0,1.05,3.00,1.74,1.00,300,0.26,GEOLABNOR
2,6406/2-1,QUATERNARY,302,1240,938,0.0,65.0,2530.000,SPRINGAR FM,DC,SH/CLYST,360.0,1.85,4.23,2.54,1.01,419,0.30,GEOLABNOR
3,6406/2-1,QUATERNARY,302,1240,938,0.0,65.0,2540.000,SPRINGAR FM,DC,CLYST/SH,360.0,1.34,2.97,1.72,0.80,371,0.31,GEOLABNOR(SAGA)
4,6406/2-1,QUATERNARY,302,1240,938,0.0,65.0,2540.001,SPRINGAR FM,DC,SH/CLYST,360.0,1.34,2.97,1.72,0.80,371,0.31,GEOLABNOR


In [14]:
merge = merge [ (merge['Depth m RKB'] > merge['Top_depth_m']) & (merge['Depth m RKB'] <= merge['Base_depth_m']) ]

In [15]:
len(merge.index)

303

In [16]:
merge.sample(10)

,Well,Period,Top_depth_m,Base_depth_m,Thickness_m,Top_age_Ma,Bottom_age_Ma,Depth m RKB,Formation (NPD),Type of measurement,samplith,tmax,s1,s2,s3,toc,HI,pi,company_name
6781,6406/2-2,CONIACIAN,3070,3340,270,86.6,88.9,3170.0,SHETLAND,DC,CLYST/SH,430.0,27.34,3.67,0.50,1.37,268,0.88,GEOLABNOR(SAGA)
9616,6406/3-1,UPPER CRETACEOUS,2620,3205,585,65.0,97.0,3200.0,SHETLAND,DC,MDST,434.0,0.09,0.77,0.62,0.96,80,0.10,GEOCHEM(SAGA)
2681,6406/2-1,LOWER TURONIAN,3450,3960,510,89.8,92.6,3465.0,LYSING FM,DC,CLYST/SH,362.0,1.72,4.66,1.64,1.44,324,0.27,GEOLABNOR(SAGA)
1932,6406/2-1,MIDDLE CAMPANIAN,2610,2690,80,77.0,80.0,2665.0,SPRINGAR FM,DC,CLYST/SH,351.0,1.51,3.17,1.58,1.08,294,0.32,GEOLABNOR(SAGA)
2222,6406/2-1,SANTONIAN,2848,3070,222,83.0,86.6,2865.0,SHETLAND,DC,CLYST/SH,353.0,2.21,3.96,1.78,1.04,381,0.36,GEOLABNOR(SAGA)
6520,6406/2-2,LOWER CAMPANIAN,2710,2910,200,78.5,83.0,2910.0,SHETLAND,SWC,SLST,435.0,4.19,2.00,NaN,1.16,172,0.68,SAGA(SAGA)
6885,6406/2-2,MIDDLE TURONIAN,3340,3600,260,88.9,90.4,3590.0,LANGE UP FM,DC,CLYST/SH,428.0,57.12,6.80,0.97,1.64,415,0.89,GEOLABNOR(SAGA)
9595,6406/3-1,UPPER CRETACEOUS,2620,3205,585,65.0,97.0,2800.0,SHETLAND,DC,MDST,429.0,0.06,0.70,1.09,0.93,75,0.08,GEOCHEM(SAGA)
7073,6406/2-2,MIDDLE CENOMANIAN,4040,4215,175,92.6,97.0,4120.0,LANGE UP FM,DC,CLYST/SH,434.0,38.27,3.76,0.78,1.89,199,0.91,GEOLABNOR(SAGA)
6878,6406/2-2,MIDDLE TURONIAN,3340,3600,260,88.9,90.4,3453.0,LYSING FM,SWC,CLYST/SST,438.0,4.90,2.10,NaN,1.42,148,0.70,SAGA(SAGA)


In [17]:
merge.drop(['Top_depth_m', 'Base_depth_m', 'Thickness_m','Top_age_Ma', 'Bottom_age_Ma'], axis=1, inplace=True)

In [21]:
#merge = merge.reset_index(inplace=True)
merge.index = pd.RangeIndex(len(merge.index))

In [23]:
merge.index

RangeIndex(start=0, stop=303, step=1)

In [22]:
merge.head()

,Well,Period,Depth m RKB,Formation (NPD),Type of measurement,samplith,tmax,s1,s2,s3,toc,HI,pi,company_name
0,6406/2-1,UPPER CAMPANIAN,2510.000,SPRINGAR FM,DC,SH/CLYST,356.0,2.20,4.61,2.14,1.15,401,0.32,GEOLABNOR
1,6406/2-1,UPPER CAMPANIAN,2520.000,SPRINGAR FM,DC,SH/CLYST,362.0,1.05,3.00,1.74,1.00,300,0.26,GEOLABNOR
2,6406/2-1,UPPER CAMPANIAN,2530.000,SPRINGAR FM,DC,SH/CLYST,360.0,1.85,4.23,2.54,1.01,419,0.30,GEOLABNOR
3,6406/2-1,UPPER CAMPANIAN,2540.000,SPRINGAR FM,DC,CLYST/SH,360.0,1.34,2.97,1.72,0.80,371,0.31,GEOLABNOR(SAGA)
4,6406/2-1,UPPER CAMPANIAN,2540.001,SPRINGAR FM,DC,SH/CLYST,360.0,1.34,2.97,1.72,0.80,371,0.31,GEOLABNOR


In [24]:
with pd.ExcelWriter('combine.xlsx') as writer:  # doctest: +SKIP
    merge.to_excel(writer, sheet_name='combined')